<a href="https://colab.research.google.com/github/ashok426/Querying-sqlite-from-python/blob/main/SQL_on_imdb_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Querying SQLite from python to access imdb movie dataset 

In [ ]:
pwd

'/content'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

### Once we import the module, we connect to the database we want to query using the connect() function.

In [ ]:
conn = sqlite3.connect("/content/drive/MyDrive/Applied AI/assignment/imdb sql assignment/Db-IMDB-Assignment.db")

#### Overview of all tables

In [ ]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)


In [ ]:
tables

,Table_Name
0,Movie
1,Genre
2,Language
3,Country
4,Location
5,M_Location
6,M_Country
7,M_Language
8,M_Genre
9,Person


In [ ]:
tables = tables["Table_Name"].values.tolist()

In [ ]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [ ]:
#answer for 1st question

%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    print(q1_results.shape)
    


query1 = """ SELECT s.movie_name, s.year, s.director
FROM(SELECT mo.title movie_name, CAST(SUBSTR(TRIM(mo.year),-4) AS INTEGER) year,p.Name director,TRIM(g.Name) genre,
CASE
WHEN ((mo.year%4=0) AND (mo.year%100!=0)) OR (mo.year%400=0) THEN "LEAP"
ELSE "NON LEAP"
END leapyear
FROM M_genre m
INNER JOIN Movie mo ON mo.MID = m.MID
INNER JOIN Genre g ON g.GID = m.GID
INNER JOIN M_Director md ON md.MID = mo.MID
INNER JOIN Person p ON md.PID = p.PID
WHERE leapyear = "LEAP") s
WHERE s.genre LIKE "%Comedy%";
"""
grader_1(query1)

                          movie_name  year           director
0                         Mastizaade  2016       Milap Zaveri
1  Harold & Kumar Go to White Castle  2004       Danny Leiner
2                 Gangs of Wasseypur  2012     Anurag Kashyap
3        Around the World in 80 Days  2004       Frank Coraci
4             The Accidental Husband  2008      Griffin Dunne
5                             Barfi!  2012        Anurag Basu
6                  Bride & Prejudice  2004    Gurinder Chadha
7    Beavis and Butt-Head Do America  1996         Mike Judge
8                          Blackmail  2018        Abhinay Deo
9                            Dostana  2008   Tarun Mansukhani
(246, 3)
CPU times: user 63.3 ms, sys: 8.71 ms, total: 72 ms
Wall time: 80.5 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [ ]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    


query2 = """ 
SELECT trim(p.Name)
FROM Person p
WHERE p.PID IN (
select trim(c.PID) PD
from M_cast c 
where c.MID in (
    select mo.MID from Movie mo where CAST(SUBSTR(mo.year,-4) AS Integer) = 1971 AND mo.title = "Anand"
)
);
  """

grader_2(query2)

       trim(p.Name)
0  Amitabh Bachchan
1     Rajesh Khanna
2     Sumita Sanyal
3        Ramesh Deo
4         Seema Deo
5    Asit Kumar Sen
6        Dev Kishan
7      Atam Prakash
8     Lalita Kumari
9            Savita
CPU times: user 26.4 ms, sys: 3.76 ms, total: 30.1 ms
Wall time: 41.8 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
%%time

def grader_3a(query_less_1970, query_more_1990):
    q3_a = pd.read_sql_query(query_less_1970,conn)
    print(q3_a.shape)
    q3_b = pd.read_sql_query(query_more_1990,conn)
    print(q3_b.shape)
    
query_less_1970 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)<1970
)
) r1 
on r1.PD=p.PID 
"""
query_more_1990 =""" 
Select p.PID from Person p 
inner join 
(
    select trim(mc.PID) PD, mc.MID from M_cast mc 
where mc.MID 
in 
(
    select mv.MID from Movie mv where CAST(SUBSTR(mv.year,-4) AS Integer)>1990
)
) r1 
on r1.PD=p.PID """
print(grader_3a(query_less_1970, query_more_1990))

# using the above two queries, you can find the answer to the given question 

(4942, 1)
(62570, 1)
True
CPU times: user 248 ms, sys: 6.91 ms, total: 255 ms
Wall time: 257 ms


In [ ]:
#answer for question 3

%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    print(q3_results.shape)
    

query3 = """
SELECT p.Name
FROM Person p
WHERE p.PID IN (
SELECT squ.PDS
FROM (SELECT trim(c.PID) PD, c.MID
FROM M_Cast c
WHERE c.MID IN 
(
select mo.MID from Movie mo where CAST(SUBSTR(mo.year,-4) AS Integer)<1970 
)
) sq 
INNER JOIN 
(
  SELECT trim(c.PID) PDS, c.MID
FROM M_Cast c
WHERE c.MID IN 
(
select mo.MID from Movie mo where CAST(SUBSTR(mo.year,-4) AS Integer)>1990 
)
)squ ON sq.PD = squ.PDS);
"""
grader_3(query3)

                Name
0       Rishi Kapoor
1   Amitabh Bachchan
2             Asrani
3       Zohra Sehgal
4    Parikshat Sahni
5      Rakesh Sharma
6        Sanjay Dutt
7          Ric Young
8              Yusuf
9     Suhasini Mulay
(300, 1)
CPU times: user 3min 21s, sys: 4.77 s, total: 3min 26s
Wall time: 3min 25s


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [ ]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    
#query_4a =""" *** Write a query, which will return all the directors(id's) along with the number of movies they directed *** """
query_4a = """
SELECT d.PID, COUNT(*)
FROM M_Director d 
INNER JOIN Movie m ON m.MID = d.MID
GROUP BY d.PID  
ORDER BY COUNT(*) DESC;
"""
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

         PID  COUNT(*)
0  nm0223522        39
1  nm0080315        35
2  nm0698184        30
3  nm0890060        30
4  nm0080333        29
5  nm0611531        27
6  nm0007181        21
7  nm0154113        19
8  nm0759662        19
9  nm0007131        18
True
CPU times: user 18.1 ms, sys: 0 ns, total: 18.1 ms
Wall time: 23.6 ms


In [ ]:
#answer for question 4

%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    print(q4_results.shape)
    

query4 = """
SELECT p.Name Director_name, COUNT(*) no_of_movies
FROM M_Director d 
INNER JOIN Movie m ON m.MID = d.MID
INNER JOIN Person p ON p.PID = d.PID
GROUP BY p.Name 
HAVING COUNT(*) >= 10
ORDER BY COUNT(*) DESC;
""" 
grader_4(query4)

           Director_name  no_of_movies
0           David Dhawan            39
1           Mahesh Bhatt            36
2        Ram Gopal Varma            30
3           Priyadarshan            30
4           Vikram Bhatt            29
5   Hrishikesh Mukherjee            27
6            Yash Chopra            21
7        Basu Chatterjee            19
8         Shakti Samanta            19
9           Subhash Ghai            18
(58, 2)
CPU times: user 68 ms, sys: 283 µs, total: 68.2 ms
Wall time: 69 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    

query_5aa =""" 
SELECT sub.movie_id, TRIM(p.Gender), COUNT(*)
FROM (SELECT TRIM(mo.MID) movie_id, TRIM(c.PID) per_id
FROM M_Cast c
INNER JOIN Movie mo ON mo.MID=c.MID) sub
INNER JOIN Person p ON p.PID=sub.per_id
GROUP BY 1, 2;
 """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    

query_5ab = """ 
SELECT sub.movie_id mov_id, TRIM(p.Gender) gender, COUNT(*)
FROM (SELECT TRIM(mo.MID) movie_id, TRIM(c.PID) per_id
FROM M_Cast c
INNER JOIN Movie mo ON mo.MID=c.MID) sub
INNER JOIN Person p ON p.PID=sub.per_id
GROUP BY 1, 2
HAVING gender = "Male";
"""

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

    movie_id TRIM(p.Gender)  COUNT(*)
0  tt0021594           None         1
1  tt0021594         Female         3
2  tt0021594           Male         5
3  tt0026274           None         2
4  tt0026274         Female        11
5  tt0026274           Male         9
6  tt0027256           None         2
7  tt0027256         Female         5
8  tt0027256           Male         8
9  tt0028217         Female         3
True
      mov_id gender  COUNT(*)
0  tt0021594   Male         5
1  tt0026274   Male         9
2  tt0027256   Male         8
3  tt0028217   Male         7
4  tt0031580   Male        27
5  tt0033616   Male        46
6  tt0036077   Male        11
7  tt0038491   Male         7
8  tt0039654   Male         6
9  tt0040067   Male        10
True
CPU times: user 527 ms, sys: 17.3 ms, total: 544 ms
Wall time: 547 ms


In [ ]:
# answer for question 5a.

%%time
def grader_5a(q5a):
    q5a_results  = pd.read_sql_query(q5a,conn)
    print(q5a_results.head(10))
    

query5a = """ 
SELECT year, COUNT(*) Female_Cast_Only_Movies
FROM Movie 
WHERE MID NOT IN (SELECT sub.movie_id mov_id
FROM (SELECT TRIM(mo.MID) movie_id, TRIM(c.PID) per_id
FROM M_Cast c
INNER JOIN Movie mo ON mo.MID=c.MID) sub
INNER JOIN Person p ON p.PID=sub.per_id
GROUP BY 1, TRIM(p.Gender)
HAVING TRIM(p.Gender) = "Male")
GROUP BY 1 
 """
grader_5a(query5a)

     year  Female_Cast_Only_Movies
0    1939                        1
1    1999                        1
2    2000                        1
3  I 2018                        1
CPU times: user 219 ms, sys: 78 µs, total: 219 ms
Wall time: 226 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [ ]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    

query5b =  """ 
SELECT year, COUNT(*) Total_Movies, (SELECT COUNT(*) Female_Cast_Only_Movies
FROM Movie 
WHERE MID NOT IN (SELECT sub.movie_id mov_id
FROM (SELECT TRIM(mo.MID) movie_id, TRIM(c.PID) per_id
FROM M_Cast c
INNER JOIN Movie mo ON mo.MID=c.MID) sub
INNER JOIN Person p ON p.PID=sub.per_id
GROUP BY 1, TRIM(p.Gender)
HAVING TRIM(p.Gender) = "Male")
GROUP BY year ) Female_movies
FROM Movie
WHERE year IN
(SELECT year
FROM Movie 
WHERE MID NOT IN (SELECT sub.movie_id mov_id
FROM (SELECT TRIM(mo.MID) movie_id, TRIM(c.PID) per_id
FROM M_Cast c
INNER JOIN Movie mo ON mo.MID=c.MID) sub
INNER JOIN Person p ON p.PID=sub.per_id
GROUP BY 1, TRIM(p.Gender)
HAVING TRIM(p.Gender) = "Male")
GROUP BY 1) 
GROUP BY 1
 """
grader_5b(query5b)

     year  Total_Movies  Female_movies
0    1939             2              1
1    1999            66              1
2    2000            64              1
3  I 2018            10              1
CPU times: user 424 ms, sys: 7.19 ms, total: 431 ms
Wall time: 437 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [ ]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    print(q6_results.shape)
    #assert (q6_results.shape == (3473, 2))

query6 = """
SELECT TRIM(mo.title), COUNT(DISTINCT TRIM(c.PID)) cast_size 
FROM Movie mo
INNER JOIN M_Cast c ON c.MID = mo.MID
GROUP BY 1
ORDER BY 2 DESC;
"""
grader_6(query6)

               TRIM(mo.title)  cast_size
0               Ocean's Eight        238
1                    Apaharan        233
2                        Gold        215
3             My Name Is Khan        213
4  Captain America: Civil War        191
5                    Geostorm        170
6                     Striker        165
7                        2012        154
8                      Pixels        144
9       Yamla Pagla Deewana 2        140
(3342, 2)
CPU times: user 182 ms, sys: 17 ms, total: 199 ms
Wall time: 208 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [ ]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

query8a = """ 
SELECT TRIM(c.PID) actor, TRIM(d.PID) director, COUNT(*) movies
FROM Movie mo
INNER JOIN M_Director d ON d.MID = mo.MID
INNER JOIN M_Cast c ON c.MID = mo.MID
GROUP BY 1, 2;
"""
grader_8a(query8a)

# using the above query, you can write the answer to the given question

       actor   director  movies
0  nm0000002  nm0496746       1
1  nm0000027  nm0000180       1
2  nm0000039  nm0896533       1
3  nm0000042  nm0896533       1
4  nm0000047  nm0004292       1
5  nm0000073  nm0485943       1
6  nm0000076  nm0000229       1
7  nm0000092  nm0178997       1
8  nm0000093  nm0000269       1
9  nm0000096  nm0113819       1
CPU times: user 357 ms, sys: 23.9 ms, total: 381 ms
Wall time: 387 ms


In [ ]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    #assert (q8_results.shape == (245, 2))

query8 = """
SELECT TRIM(pe.name) actor, ca.director, COUNT(ca.director) no_of_times
FROM (SELECT TRIM(c.PID) actor_id, TRIM(p.Name) director, COUNT(*) movies
FROM Movie mo
INNER JOIN M_Director d ON d.MID = mo.MID
INNER JOIN M_Cast c ON c.MID = mo.MID
INNER JOIN Person p ON p.PID= d.PID
GROUP BY 1, 2) ca
INNER JOIN Person pe ON pe.PID=ca.actor_id
GROUP BY 1,2

"""
grader_8(query8)

                        actor            director  no_of_times
0             'Ganja' Karuppu       N. Linguswamy            1
1             'Ganja' Karuppu            Perarasu            1
2       'Lee' George Quinones    Adam Bhala Lough            1
3  'Musafir' Radio Performing     Abhishek Kapoor            1
4          'Nandha' Saravanan                Bala            1
5      'Om' Rakesh Chaturvedi            R. Balki            1
6              'Snub' Pollard         Robert Wise            1
7              A'Ali de Sousa         Karan Johar            1
8             A. Abdul Hameed    K.S. Prakash Rao            1
9             A. Abdul Hameed  K.S. Sethumadhavan            1
(73292, 3)
CPU times: user 716 ms, sys: 39.7 ms, total: 755 ms
Wall time: 756 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [ ]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    #assert (q9a_results.shape == (2382, 1))

query9a = """ 
SELECT DISTINCT(c.PID) cast
FROM M_Cast c
WHERE TRIM(c.MID) IN 
(SELECT TRIM(c.MID)
FROM M_Cast c
INNER JOIN Person p ON TRIM(p.PID)=TRIM(c.PID)
WHERE p.Name LIKE "%Shah Rukh Khan%");

"""
grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

         cast
0   nm0451321
1   nm0004418
2   nm1995953
3   nm2778261
4   nm0631373
5   nm0241935
6   nm0792116
7   nm1300111
8   nm0196375
9   nm1464837
(2383, 1)
CPU times: user 60.4 ms, sys: 7.84 ms, total: 68.2 ms
Wall time: 77.2 ms


In [ ]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    #assert (q9_results.shape == (25698, 1))

query9 = """ 
SELECT DISTINCT(TRIM(cas.PID)) co_actor_id
FROM M_Cast cas
WHERE cas.MID IN 
(
SELECT TRIM(ca.MID) mov_id 
FROM M_Cast ca
INNER JOIN 
(
SELECT DISTINCT(TRIM(c.PID)) cast
FROM M_Cast c
WHERE TRIM(c.MID) IN 
(SELECT TRIM(c.MID)
FROM M_Cast c
INNER JOIN Person p ON TRIM(p.PID)=TRIM(c.PID)
WHERE p.Name LIKE "%Shah Rukh Khan%")
) co_ac ON TRIM(ca.PID) = co_ac.cast
) 


"""
grader_9(query9)

  co_actor_id
0   nm2539953
1   nm0922035
2   nm0324658
3   nm0943079
4   nm0000218
5   nm0001394
6   nm0929654
7   nm3116102
8   nm3248891
9   nm2418809
(28081, 1)
CPU times: user 208 ms, sys: 9.38 ms, total: 217 ms
Wall time: 221 ms
